In [8]:
import kagglehub
import pandas as pd
# Download latest version
path = kagglehub.dataset_download("meetnagadia/apple-stock-price-from-19802021")

print(path)

C:\Users\justin.beresford\.cache\kagglehub\datasets\meetnagadia\apple-stock-price-from-19802021\versions\3


In [13]:
df =pd.read_csv("C:/Users/justin.beresford/.cache/kagglehub/datasets/meetnagadia/apple-stock-price-from-19802021/versions/3/AAPL.csv")

In [17]:
df = df.sort_index()  # ensure data is sorted by date

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Use only the 'Adj Close' column for modeling (or 'Close' if Adj Close not available)
data = df[["Adj Close"]].copy()
# Scale prices to [0, 1] range
scaler = MinMaxScaler(feature_range=(0, 1))
data["Adj Close"] = scaler.fit_transform(data[["Adj Close"]])

In [ ]:
import numpy as np


data
sequence_length = 60
prices = data["Adj Close"].values

X = []
y = []

for i in range(sequence_length, len(prices)):
    X.append(prices[i-sequence_length:i])
    y.append(prices[i])
X = np.array(X)
y = np.array(y)
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (10408, 60)
Shape of y: (10408,)


In [25]:
X = X.reshape((X.shape[0], X.shape[1], 1))
print("Reshaped X:", X.shape)  # e.g., (N, 60, 1)

Reshaped X: (10408, 60, 1)


In [28]:
# Simple train-test split (time-based)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [31]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(sequence_length, 1)))  # single LSTM layer
model.add(Dense(1))  # output layer for price prediction

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=20, batch_size=32, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)


In [ ]:
y_pred = model.predict(X_test)

# Invert scaling to get actual price values
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred)
